In [ ]:
import json
import numpy as np
from glob import glob

from IPython.display import HTML

from radiopadre import ls                                                # everyone needs an ls function
from radiopadre import settings
from radiopadre.table import tabulate
from radiopadre.layouts import Title, Section

from bokeh.io import output_notebook
from bokeh.layouts import row, column
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool, LinearAxis, Range1d

output_notebook()
settings.html.width = 800
settings.html.height = 800
Title("CARACal Data Reduction",                                          # edit the title as you like
      "Observation info | Fields Info | Flagging Data | Gain Plots | Data Inspection | Cross-Cal flagging | "
      "Target flagging | Target RFInder | Continuum Images | Calibration Diagnostics |  Self-Cal flagging | "
      "Continuum Statistics | Continuum Cross-matching | Sun Blocker | Cube Images | Moment Images | "
      "Spectral Analysis Logs")

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
def plot_flagged_summary(summary_files, main_separator=' flagged:'):
    try:
        plots={
               'field ': {'x_label': 'Field', 'y_label': 'Flagged data (%)', 'title':'Field RFI summary', 'rotate_xlabel':False},
               'antenna ': {'x_label': 'Antenna', 'y_label': 'Flagged data (%)', 'title':'Antenna RFI summary', 'rotate_xlabel':True},
               'scan ': {'x_label': 'Scans', 'y_label': 'Flagged data (%)', 'title':'Scans RFI summary', 'rotate_xlabel':True},
               'correlation ': {'x_label': 'Correlation', 'y_label': 'Flagged data (%)', 'title':'Correlation RFI summary', 'rotate_xlabel':False}
              }

        fs = glob(summary_files)
        with open(fs[0]) as f:
            data = f.readlines()

        plot_list = []
        for plot_key in plots.keys():
            keys = []
            flagged = []
            for d in data:
                if plot_key in d:
                    value = d.split(plot_key)[1].split(main_separator)[0]
                    if value not in keys:
                        keys.append(value)
                        flagged.append(float(d.split()[-1][1:-2]))

            if len(keys) > 0:
                if plot_key=='scan ':
                    zipped_lists = zip(list(map(int, keys)), flagged)
                    keys, flagged = zip(*sorted(zipped_lists))
                    keys = [str(key) for key in keys]

                rotate_xlabel=plots[plot_key]['rotate_xlabel']
                x_label=plots[plot_key]['x_label']
                y_label=plots[plot_key]['y_label']
                title=plots[plot_key]['title']
                plotter = figure(x_range=keys, x_axis_label=x_label, y_axis_label=y_label,
                                 plot_width=400, plot_height=300, title=title)

                plotter.vbar(x=keys, top=flagged, width=0.9)

                plotter.xgrid.grid_line_color = None
                plotter.y_range.start = 0
                plotter.title.align = 'center'
                if rotate_xlabel:
                    plotter.xaxis.major_label_orientation = np.pi/2
                plot_list.append(plotter)
        if len(plot_list) == 4:
            show(column(row(plot_list[0], plot_list[3]),
                        row(plot_list[2], plot_list[1])))
        else:
            return HTML("<p style='color:#ff0000; font-weight:bold'>Could not load specified files. "
                            "This step may not have run, or there was a trainsmash. Check the logs</p>")
    except IndexError:
        return HTML("<p style='color:#ff0000; font-weight:bold'>Could not load specified files. "
                    "This step may not have run, or there was a trainsmash. Check the logs</p>")

def plot_aimfast_stats(filename='fidelity_results.json'):
    if glob(filename):
        with open(filename) as f:  
            data = json.load(f)
        res_stats = dict()
        dr_stats = dict()
        for par, val in data.items():
            if '.fits' not in par and 'models' not in val and type(val) is not list:
                for p, v in val.items():
                    if type(v) is dict:
                        dr_stats[p] = v
                        val_copy = val.copy()
                        val_copy.pop(p)
                res_stats[par] = val_copy
                res_stats[par]['NORM'] = res_stats[par]['NORM'][0]

        res_stats = dict(sorted(res_stats.items()))
        dr_stats = dict(sorted(dr_stats.items()))
        
        im_keys = []
        rms_values = []
        stddev_values = []
        mad_values = []
        slidingdev_values = []
        skew_values = []
        kurt_values = []
        norm_values = []
        for res_stat in res_stats:
            im_keys.append(res_stat.replace('-residual', ''))
            rms_values.append(res_stats[res_stat]['RMS'])
            stddev_values.append(res_stats[res_stat]['STDDev'])
            mad_values.append(res_stats[res_stat]['MAD'])
            slidingdev_values.append(res_stats[res_stat]['SLIDING_STDDev'])
            skew_values.append(res_stats[res_stat]['SKEW'])
            kurt_values.append(res_stats[res_stat]['KURT'])
            norm_values.append(res_stats[res_stat]['NORM'])

        width = 400
        height = 300
        multiplier = 10e6 # micro

        # Vriance plots
        variance_plotter = figure(x_range=im_keys, x_axis_label="Image", y_axis_label="Flux density (µJy)",
                                  plot_width=width, plot_height=height, title='Residual Variance')
        variance_plotter.line(im_keys, np.array(stddev_values)*multiplier, legend_label='std', color='blue')
        variance_plotter.line(im_keys, np.array(mad_values)*multiplier, legend_label='mad', color='red')
        variance_plotter.line(im_keys, np.array(slidingdev_values)*multiplier, legend_label='sliding', color='green')
        variance_plotter.title.align = 'center'

        # Moment 3 & 4 plots
        mom34_plotter = figure(x_range=im_keys, x_axis_label="Image", y_axis_label="Value",
                               plot_width=width, plot_height=height, title='Skewness & Kurtosis')
        mom34_plotter.line(im_keys, skew_values, legend_label='Skewness', color='blue')
        mom34_plotter.line(im_keys, kurt_values, legend_label='kurtosis', color='red')
        mom34_plotter.title.align = 'center'
        
        # Normality test plot
        normalised = np.array(norm_values)/norm_values[0]
        norm_plotter = figure(x_range=im_keys, x_axis_label="Image", y_axis_label="Value",
                              plot_width=width, plot_height=height, title='Normality Tests')
        norm_plotter.vbar(x=im_keys, top=normalised, width=0.9)
        #norm_plotter.y_range.start = 0
        norm_plotter.title.align = 'center'

        # Dynamic Range plot
        dr_keys = []
        dr_values = []
        for dr_stat in dr_stats:
            dr_keys.append(dr_stat.replace('-model', ''))
            dr_values.append(dr_stats[dr_stat]['DR'])
        dr_plotter = figure(x_range=dr_keys, x_axis_label="Image", y_axis_label="Value",
                            plot_width=width, plot_height=height, title='Dynamic Range')
        dr_plotter.vbar(x=dr_keys, top=dr_values, width=0.9)
        #dr_plotter.y_range.start = 0
        dr_plotter.title.align = 'center'
        return (show(column(row(variance_plotter, mom34_plotter),
                        row(norm_plotter, dr_plotter))),)
    else:
        return (HTML("<p style='color:#ff0000; font-weight:bold'>Could not load specified files. "
                    "This step may not have run, or there was a trainsmash. Check the logs</p>"),)

In [ ]:
obsinfo = ls("*-obsinfo.txt", section="Observation info")
obsinfo.title = "Observation Information"
if obsinfo:
    obsinfo[-1][0:8].show(number=False)
else:
    HTML("<p style='color:#ff0000; font-weight:bold'>Could not load specified files. "
         "This step may not have run, or there was a trainsmash. Check the logs</p>")

In [ ]:
elev_plot = ls("diagnostic_plots/*elevation-tracks*.png", section="Fields Info")
elev_plot.title = "Elevation Plot"
if obsinfo:
    start_line = obsinfo[-1]('Fields')[0][0]+1
    end_line = obsinfo[-1]('Spectral')[0][0]
    get_sources = obsinfo[-1][start_line:end_line].extract(
                      "([^\s]+)\s+([^\s]+)\s+([^\s]+)\s+([^\s]+)\s+([^\s]+)\s+([^\s]+)\s+([^\s]+)", 
                      [0, 2, 3, 4])
else:
    get_sources = "No observation info file"
tabulate([elev_plot.thumbs, get_sources], cw=[0.6, 0.4], maxcol=2)

In [ ]:
summary_file = "logs*/log-flagging-flagging_summary_flagging_0__0-*.txt"
rfi_summary = ls(summary_file, section="Flagging Data")
plot_flagged_summary(summary_file)

In [ ]:
gain_files = "reports/*1gc*"
gain_plots = ls(gain_files, section="Gain Plots")
gain_plots.html.thumbs

In [ ]:
inspect_files = "diagnostic_plots/crosscal/*.png"
inspect_plots = ls("{} -rt".format(inspect_files), section="Data Inspection")
inspect_plots.thumbs

In [ ]:
summary_file = "logs*/log-cross_cal-flagging_summary_crosscal_1gc1_0*.txt"
rfi_summary = ls(summary_file, section="Cross-Cal flagging")
plot_flagged_summary(summary_file)

In [ ]:
summary_file = "logs*/log-flagging-flagging_summary_flagging__2_0_*.txt"
rfi_summary = ls(summary_file, section="Target flagging")
plot_flagged_summary(summary_file)

In [ ]:
rfinder_file = "rfi_q*/time_report.html"
rfinder_plot = ls(rfinder_file, section="Target RFInder")
rfinder_plot.html.thumbs(width=1200,height=1000)

In [ ]:
image_files = "continuum/image*/*MFS-image.fits"
image_plots = ls(image_files, section="Continuum Images")
image_plots.thumbs(vmin=-0.0002, vmax=0.002, maxcol=5)

In [ ]:
residual_files = "continuum/image*/*MFS-residual.fits"
residual_plots = ls(residual_files)
residual_plots.thumbs(vmin=-0.0001, vmax=0.001, maxcol=5)

In [ ]:
model_files = "continuum/image*/*MFS-model.fits"
model_plots = ls(model_files)
model_plots.thumbs(vmin=-0.00002, vmax=0.00002, maxcol=5)

In [ ]:
psf_files = "continuum/image*/*MFS-psf.fits"
psf_plots = ls(psf_files)
psf_plots.thumbs(vmin=-0.0002, vmax=0.002, maxcol=5)

In [ ]:
image_files = "continuum/image*/*MFS-dirty.fits"
image_plot = ls(image_files)
image_plot.thumbs(vmin=-0.0002, vmax=0.002, maxcol=5)

In [ ]:
calibration_files = "continuum/selfcal_products/*png"
calibration_plots = ls(calibration_files, section="Calibration Diagnostics")
calibration_plots.thumbs(maxcol=6)

In [ ]:
summary_files = "logs*/log-self_cal-flagging_summary_image_selfcal_0*.txt"
rfi_summary = ls(summary_files, section="Self-Cal flagging")
plot_flagged_summary(summary_files)

In [ ]:
fidelity_file = "fidelity_results.json"
fidelity = ls(fidelity_file, section="Continuum Statistics")
plot_aimfast_stats(filename=fidelity_file)[0]

In [ ]:
cross_match_files = "diagnostic_plots/selfcal/*html"
cross_match_plot = ls(cross_match_files, section="Continuum Cross-matching")
cross_match_plot.thumbs()

In [ ]:
sunblocker_files = "diagnostic_plots/*sunblocker.svg"
sunblocker_plot = ls(sunblocker_files, section="Sun Blocker")
sunblocker_plot.thumbs()

In [ ]:
cube_image_files = "cubes/cube*/*[*image|*residual].fits"
cube_image_plots = ls(cube_image_files, section="Cube Images")
cube_image_plots.thumbs(vmin=-0.0001, vmax=0.001)

In [ ]:
moment0_image_files = "cubes/cube*/*image_mom0.fits"
moment0_image_plots = ls(moment0_image_files, section="Moment Images")
moment0_image_plots.thumbs(vmin=-0.02, vmax=0.005)

In [ ]:
moment1_image_files = "cubes/cube*/*image_mom1.fits"
moment1_image_plots = ls(moment1_image_files)
moment1_image_plots.thumbs(vmin=1e9, vmax=2e9)

In [ ]:
sharpener_file = "diagnostic_plots/wlm_J0001_1527_sharpOut/plot/output_all_plots_*.pdf"
sharpener_plots = ls(sharpener_file, section="Spectral Analysis")
sharpener_plots.thumbs()